# Homework 3 - Benjamin Case - 4/16/19
Solve a maximum flow problem for a directed graph with no loops or multiedges. 

We implemented a preflow-push (i.e. push-relabel) algorithm that uses a max heap on active node excess to improve efficiency. Input is in the format of a NetGen problem. 

Code Sources:
The max heap code is from: https://github.com/DanielStutzbach/heapdict/blob/master/heapdict.py. 
The graph data structure code accompanies the Goodrich, Tamassia, Goldwasser textbook. We have made a couple modifications including adding the set_element() methods to the Vertex and Edge classes. 


In [12]:
# Heap
# https://github.com/DanielStutzbach/heapdict/blob/master/heapdict.py
import collections

def doc(s):
    if hasattr(s, '__call__'):
        s = s.__doc__
    def f(g):
        g.__doc__ = s
        return g
    return f

class heapdict(collections.MutableMapping):
    __marker = object()

    @staticmethod
    def _parent(i):
        return ((i - 1) >> 1)

    @staticmethod
    def _left(i):
        return ((i << 1) + 1)

    @staticmethod
    def _right(i):
        return ((i+1) << 1)    
    
    def __init__(self, *args, **kw):
        self.heap = []
        self.d = {}
        self.update(*args, **kw)

    @doc(dict.clear)
    def clear(self):
        del self.heap[:]
        self.d.clear()

    @doc(dict.__setitem__)
    def __setitem__(self, key, value):
        if key in self.d:
            self.pop(key)
        wrapper = [value, key, len(self)]
        self.d[key] = wrapper
        self.heap.append(wrapper)
        self._decrease_key(len(self.heap)-1)

    def _min_heapify(self, i):
        l = self._left(i)
        r = self._right(i)
        n = len(self.heap)
        if l < n and self.heap[l][0] < self.heap[i][0]:
            low = l
        else:
            low = i
        if r < n and self.heap[r][0] < self.heap[low][0]:
            low = r

        if low != i:
            self._swap(i, low)
            self._min_heapify(low)

    def _decrease_key(self, i):
        while i:
            parent = self._parent(i)
            if self.heap[parent][0] < self.heap[i][0]: break
            self._swap(i, parent)
            i = parent

    def _swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        self.heap[i][2] = i
        self.heap[j][2] = j

    @doc(dict.__delitem__)
    def __delitem__(self, key):
        wrapper = self.d[key]
        while wrapper[2]:
            parentpos = self._parent(wrapper[2])
            parent = self.heap[parentpos]
            self._swap(wrapper[2], parent[2])
        self.popitem()

    @doc(dict.__getitem__)
    def __getitem__(self, key):
        return self.d[key][0]

    @doc(dict.__iter__)
    def __iter__(self):
        return iter(self.d)

    def popitem(self):
        """D.popitem() -> (k, v), remove and return the (key, value) pair with lowest\nvalue; but raise KeyError if D is empty."""
        wrapper = self.heap[0]
        if len(self.heap) == 1:
            self.heap.pop()
        else:
            self.heap[0] = self.heap.pop(-1)
            self.heap[0][2] = 0
            self._min_heapify(0)
        del self.d[wrapper[1]]
        return wrapper[1], wrapper[0]    

    @doc(dict.__len__)
    def __len__(self):
        return len(self.d)

    def peekitem(self):
        """D.peekitem() -> (k, v), return the (key, value) pair with lowest value;\n but raise KeyError if D is empty."""
        return (self.heap[0][1], self.heap[0][0])

del doc
__all__ = ['heapdict']

In [13]:
# Graph
# Copyright 2013, Michael H. Goldwasser
#
# Developed for use with the book:
#
#    Data Structures and Algorithms in Python
#    Michael T. Goodrich, Roberto Tamassia, and Michael H. Goldwasser
#    John Wiley & Sons, 2013
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

class Graph:
  """Representation of a simple graph using an adjacency map."""

  #------------------------- nested Vertex class -------------------------
  class Vertex:
    """Lightweight vertex structure for a graph."""
    __slots__ = '_element'
  
    def __init__(self, x):
      """Do not call constructor directly. Use Graph's insert_vertex(x)."""
      self._element = x
  
    def element(self):
      """Return element associated with this vertex."""
      return self._element
    
    def set_element(self,_x):
      """change what is stored at the destinatin of the pointer _element"""
      self._element = _x
  
    def __hash__(self):         # will allow vertex to be a map/set key
      return hash(id(self))

    def __str__(self):
      return str(self._element)

    def __lt__(self, other):  # maybe I should take the node with greater height
      return self
    
  #------------------------- nested Edge class -------------------------
  class Edge:
    """Lightweight edge structure for a graph."""
    __slots__ = '_origin', '_destination', '_element'
  
    def __init__(self, u, v, x):
      """Do not call constructor directly. Use Graph's insert_edge(u,v,x)."""
      self._origin = u
      self._destination = v
      self._element = x
  
    def endpoints(self):
      """Return (u,v) tuple for vertices u and v."""
      return (self._origin, self._destination)
  
    def opposite(self, v):
      """Return the vertex that is opposite v on this edge."""
      if not isinstance(v, Graph.Vertex):
        raise TypeError('v must be a Vertex')
      return self._destination if v is self._origin else self._origin
      raise ValueError('v not incident to edge')
  
    def element(self):
      """Return element associated with this edge."""
      return self._element
    
    def set_element(self, _x):
        self._element = _x
        """change what is stored at the destinatin of the pointer _element"""
      
  
    def __hash__(self):         # will allow edge to be a map/set key
      return hash( (self._origin, self._destination) )

    def __str__(self):
      return '({0},{1},{2})'.format(self._origin,self._destination,self._element)
    
  #------------------------- Graph methods -------------------------
  def __init__(self, directed=False):
    """Create an empty graph (undirected, by default).

    Graph is directed if optional paramter is set to True.
    """
    self._outgoing = {}
    # only create second map for directed graph; use alias for undirected
    self._incoming = {} if directed else self._outgoing

  def _validate_vertex(self, v):
    """Verify that v is a Vertex of this graph."""
    if not isinstance(v, self.Vertex):
      raise TypeError('Vertex expected')
    if v not in self._outgoing:
      raise ValueError('Vertex does not belong to this graph.')
    
  def is_directed(self):
    """Return True if this is a directed graph; False if undirected.

    Property is based on the original declaration of the graph, not its contents.
    """
    return self._incoming is not self._outgoing # directed if maps are distinct

  def vertex_count(self):
    """Return the number of vertices in the graph."""
    return len(self._outgoing)

  def vertices(self):
    """Return an iteration of all vertices of the graph."""
    return self._outgoing.keys()

  def edge_count(self):
    """Return the number of edges in the graph."""
    total = sum(len(self._outgoing[v]) for v in self._outgoing)
    # for undirected graphs, make sure not to double-count edges
    return total if self.is_directed() else total // 2

  def edges(self):
    """Return a set of all edges of the graph."""
    result = set()       # avoid double-reporting edges of undirected graph
    for secondary_map in self._outgoing.values():
      result.update(secondary_map.values())    # add edges to resulting set
    return result

  def get_edge(self, u, v):
    """Return the edge from u to v, or None if not adjacent."""
    self._validate_vertex(u)
    self._validate_vertex(v)
    return self._outgoing[u].get(v)        # returns None if v not adjacent

  def degree(self, v, outgoing=True):   
    """Return number of (outgoing) edges incident to vertex v in the graph.

    If graph is directed, optional parameter used to count incoming edges.
    """
    self._validate_vertex(v)
    adj = self._outgoing if outgoing else self._incoming
    return len(adj[v])

  def incident_edges(self, v, outgoing=True):   
    """Return all (outgoing) edges incident to vertex v in the graph.

    If graph is directed, optional parameter used to request incoming edges.
    """
    self._validate_vertex(v)
    adj = self._outgoing if outgoing else self._incoming
    for edge in adj[v].values():
      yield edge

  def insert_vertex(self, x=None):
    """Insert and return a new Vertex with element x."""
    v = self.Vertex(x)
    self._outgoing[v] = {}
    if self.is_directed():
      self._incoming[v] = {}        # need distinct map for incoming edges
    return v
      
  def insert_edge(self, u, v, x=None):
    """Insert and return a new Edge from u to v with auxiliary element x.

    Raise a ValueError if u and v are not vertices of the graph.
    Raise a ValueError if u and v are already adjacent.
    """
    if self.get_edge(u, v) is not None:      # includes error checking
      raise ValueError('u and v are already adjacent')
    e = self.Edge(u, v, x)
    self._outgoing[u][v] = e
    self._incoming[v][u] = e

In [51]:
from collections import deque #https://docs.python.org/2/tutorial/datastructures.html, 
                             #https://docs.python.org/2/library/collections.html#collections.deque


# Read the graph from a file
file = open("graph5","r") # ****   CHANGE INPUT GRAPH  ****
for i in range(5):
    file.readline()
sourceline = file.readline()
sinkline = file.readline()
sourcelinelist = sourceline.split()
sinklinelist = sinkline.split()
n_sources = int(sourcelinelist[3])
n_sinks = int(sinklinelist[3])
print("n_sources, n_sinks:", n_sources, n_sinks)
for i in range(15):
    file.readline()
infoline = file.readline()
infolinelist = infoline.split()
n_nodes = int(infolinelist[2])
n_arcs = int(infolinelist[3])
print("n_nodes, n_arcs:", n_nodes,n_arcs)


G = Graph(True) # a directed graph
nodeList = [] # list of pointers to the Vertex objects stored in the Graph G
nodeList.append(None) # since input lables start from 1, align the off by one counting to match with the Python index

for i in range(1 ,n_nodes + 1):
    nodeList.append(G.insert_vertex((i,n_nodes, 0))) # Vertex _element = (label, height, excess)

sourcesSet = set() # set nodes that are sources
sinksSet = set() # set nodes that are sinks

# Read in sources and sinks
for i in range(n_sources + n_sinks):
    line = file.readline()
    linelist = line.split()
    nodelabel = int(linelist[1])
    source_sink = linelist[2]
    # update that node in the graph to be a source/sink
    if source_sink == "s":
        sourcesSet.add(nodeList[nodelabel])
    elif source_sink == "t":
        sinksSet.add(nodeList[nodelabel])
    else:
        raise ValueError('Inproper Input: not a source or sink label')

# read in arcs
capacityBound = 0
for i in range(n_arcs):
    linelist = file.readline().split()
    tail = int(linelist[1])
    head = int(linelist[2])
    capacity = int(linelist[3])
    capacityBound += capacity
    G.insert_edge(nodeList[tail], nodeList[head],(0,capacity)) # Edge _element = (flow, capacity)
file.close()


# perform BFS from sink(s) marking the heights (distances) of the other nodes from the nearest sink
queue = deque()
for node in sinksSet: # add the sinks to the queue first
    node.set_element(( node.element()[0], 0,node.element()[2] ))
    queue.append(node)

while True:
    try:
        node = queue.popleft() # IndexError if empty
        incoming = G.incident_edges(node,False) # the incoming edges to node
        for edge in incoming:
            # an edge has the form ((5, 6, 0),(6, 6, 0),(0, 1)), which is the edge from node 5 to 6 with capcity 1, flow 0, 
            # while both nodes have height 6 and excess 0. 
            if edge.opposite(node).element()[1] > node.element()[1] + 1: # if distance can be made smaller, do so. 
                edge.opposite(node).set_element((edge.opposite(node).element()[0],node.element()[1] + 1,0))
                queue.append(edge.opposite(node))          
    except IndexError as e: # catch when trying to pop from an empty queue which is an IndexError
        break 

# set  sources' heights back to practical "infinity" and set excesses for sources to practical "infinity"
for node in sourcesSet:
    node.set_element((node.element()[0],n_nodes,capacityBound))  # all sources have infinite supply, so infinite excess

# make the heap of active nodes and add the sources to it. 
# we will add the weights as negative to make the heap a max heap
activeHeap = heapdict()
activeHeapSet = set() # use to avoid adding redudant elements
for node in sourcesSet:
    excess = node.element()[2]
    activeHeap[node] = -excess  # add node to the active heap with weight -excess
    activeHeapSet.add(node)
    
#print("nodes after BFS: (label, height, excess)")
#for i in range(1, n_nodes+1):
#    node = nodeList[i]
#    print(node.element())
#print()

# Methods for the Main Push-Relabel (Preflow-Push) Algorithm
def push(topnode):
    """ the push operation of push-relabel algorithm """
    outgoing = G.incident_edges(topnode,True)
    # Note that: topnode.element()[2] is that nodes excess and could change during the follow for loop
    height = topnode.element()[1]
    
    for edge in outgoing:
        head = edge.opposite(topnode)
        if head.element()[1] < height:
            pushamount = min(topnode.element()[2], edge.element()[1] - edge.element()[0] ) # min(excess, capacity - flow)
            topnode.set_element((topnode.element()[0], topnode.element()[1], topnode.element()[2] - pushamount)) #update excess
            edge.set_element((edge.element()[0] + pushamount, edge.element()[1])) #update edge flow
            head.set_element(( head.element()[0], head.element()[1], head.element()[2] + pushamount )) # update head excess
            # update the head node's weight in activeHeap
            if head in activeHeapSet: # check the node is in the activeHeap already or you may add sink which should never be added
                activeHeap[head] = - head.element()[2]
            if head not in activeHeapSet and head not in sourcesSet and head not in sinksSet:
                activeHeap[head] = -head.element()[2]
                activeHeapSet.add(head)

    incoming = G.incident_edges(topnode,False)
    for edge in incoming:
        tail = edge.opposite(topnode)
        if tail.element()[1] < height:
            pushamount = min(topnode.element()[2],  edge.element()[0] ) # min(excess, flow)
            topnode.set_element((topnode.element()[0], topnode.element()[1], topnode.element()[2] - pushamount)) #update excess
            edge.set_element((edge.element()[0] - pushamount, edge.element()[1])) #update flow
            tail.set_element(( tail.element()[0], tail.element()[1], tail.element()[2] + pushamount ))
            # update the tail node's weight in activeHeap
            if tail in activeHeapSet:
                activeHeap[tail] = - tail.element()[2]
            if tail not in activeHeapSet and tail not in sourcesSet and tail not in sinksSet:
                activeHeap[tail] = -tail.element()[2]
                activeHeapSet.add(tail)

    return topnode.element()[2] # excess

def relabel(topnode):
    """ the relabel operation of the push-relabel algorithm """
    topnode.set_element((topnode.element()[0], topnode.element()[1] + 1, topnode.element()[2])) # increase the hight by 1
    newheight = topnode.element()[1]
    outgoing = G.incident_edges(topnode,True)
    incoming = G.incident_edges(topnode,False)
    needtoRelabel = True
    for edge in outgoing:
        head = edge.opposite(topnode)
        if newheight > head.element()[1]:
            needtoRelabel = False
            
    for edge in incoming:
        head = edge.opposite(topnode)
        if newheight > head.element()[1]:
            needtoRelabel = False
    
    if needtoRelabel:
        relabel(topnode)
    else:
        return

# helpful print methods
def printheap(activeHeap):
    for thing in activeHeap:
        print(thing)

def printSet(activeHeapSet):
    for thing in activeHeapSet:
        print(thing)

# Main Algorithm 
flag = True
while flag:
    #print("activeHeap")
    #printheap(activeHeap)
    try:
        (topnode, negative_excess) = activeHeap.popitem()
        activeHeapSet.remove(topnode)
        excess = topnode.element()[2]
    except  IndexError as e: 
        #print("***** Finished ****")
        flag = False
        break
    
    if topnode in sourcesSet:
        push(topnode)
    else:
        while excess > 0:
            excess = push(topnode)
            if excess > 0: 
                relabel(topnode)

# print solution    
print("Nodes after algorithm: (label, height, excess)")
for i in range(1, n_nodes+1):
    node = nodeList[i]
    print(node.element())

print("Edges after algorithm: tail, head, (flow, capacity)")
edges = G.edges()
for e in edges:
    print(e.endpoints()[0].element()[0],e.endpoints()[1].element()[0], e.element())
    
maxflow = 0
for node in sinksSet:
    maxflow += node.element()[2]
print("max flow = min cut = ", maxflow)


n_sources, n_sinks: 2 2
n_nodes, n_arcs: 400 600
Nodes after algorithm: (label, height, excess)
(1, 400, 1981303)
(2, 400, 1984563)
(3, 107, 0)
(4, 73, 0)
(5, 49, 0)
(6, 29, 0)
(7, 14, 0)
(8, 20, 0)
(9, 67, 0)
(10, 14, 0)
(11, 34, 0)
(12, 12, 0)
(13, 16, 0)
(14, 16, 0)
(15, 32, 0)
(16, 64, 0)
(17, 11, 0)
(18, 14, 0)
(19, 17, 0)
(20, 83, 0)
(21, 11, 0)
(22, 45, 0)
(23, 14, 0)
(24, 49, 0)
(25, 40, 0)
(26, 64, 0)
(27, 14, 0)
(28, 13, 0)
(29, 8, 0)
(30, 67, 0)
(31, 80, 0)
(32, 14, 0)
(33, 3, 0)
(34, 14, 0)
(35, 59, 0)
(36, 17, 0)
(37, 19, 0)
(38, 16, 0)
(39, 14, 0)
(40, 17, 0)
(41, 15, 0)
(42, 32, 0)
(43, 11, 0)
(44, 17, 0)
(45, 15, 0)
(46, 45, 0)
(47, 83, 0)
(48, 11, 0)
(49, 51, 0)
(50, 13, 0)
(51, 14, 0)
(52, 14, 0)
(53, 47, 0)
(54, 15, 0)
(55, 86, 0)
(56, 16, 0)
(57, 15, 0)
(58, 9, 0)
(59, 14, 0)
(60, 13, 0)
(61, 12, 0)
(62, 36, 0)
(63, 7, 0)
(64, 4, 0)
(65, 48, 0)
(66, 5, 0)
(67, 14, 0)
(68, 43, 0)
(69, 15, 0)
(70, 15, 0)
(71, 14, 0)
(72, 9, 0)
(73, 15, 0)
(74, 15, 0)
(75, 89, 0)
(76, 

In [5]:
G = Graph()
u = G.insert_vertex(1)
v = G.insert_vertex(2)
print(u)
#e = G.insert_edge(u,v,34)
print(e)
x = u.element()
print(x)
x = 2
print(x)
print(u.element())

u.set_element(7)
print(u.element())
e = G.insert_edge(u,v,34)

1
((366, 13, 0),(82, 12, 0),(3301, 3370))
1
2
1
7
